In [ ]:
# import libraries (make sure theyre installed)
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# read data into dataframe df and format date column
df = pd.read_csv('Series - Sheet1.csv') # put download name here
df['Date'] = pd.to_datetime(df['Date'], format='mixed')

# create bargroup dataframe with counts of games and results for bargraph
bargroup = (df.groupby(['Game', 'Result']).count()).astype(int)

# create pointgroup dataframe with counts of dates and results for timeseries graph
pointgroup = (df.groupby(['Date', 'Result']).count()).astype(int)

# ensure that all dates are present in pointgroup even if the value is 0, so that the graph is continuous
all_dates = df['Date'].unique()
all_results = df['Result'].unique()
multi_index = pd.MultiIndex.from_product([all_dates, all_results], names=['Date', 'Result'])
pointgroup = pointgroup.reindex(multi_index, fill_value=0)
pointgroup = pointgroup.reindex(df['Date'].unique(), level='Date')

# create pointgroupnodraw dataframe for analysis without draws
pointgroupnodraw = pointgroup[pointgroup.index.get_level_values('Result') != 'Draw']

# create win/loss ratio by game column in bargroup to serve as sorting metric for x axis of bottom bar graph and sort
bargroup['Win Ratio (Aidan/Oliver)'] = bargroup.apply(lambda row: row['Date'] / bargroup.loc[(row.name[0], 'Oliver Win'), 'Date'] if row.name[1] == 'Aidan Win'
                                                      else (bargroup.loc[(row.name[0], 'Aidan Win'), 'Date'] / row['Date'] if row.name[1] == 'Oliver Win' else 0), axis=1)
bargroup = bargroup.sort_values(by='Win Ratio (Aidan/Oliver)', ascending=False)

In [ ]:
# create games dataframe withh data from each game and use it to calculate win percentages by sender
games = df['Game'].unique()
win_percentages = []

for g in games:
    game_df = df[(df['Game'] == g) & (df['Result'] != 'Draw')]
    sender_wins = game_df[game_df['Sender'] == game_df['Result'].str.split().str[0]].shape[0]
    total_games = game_df.shape[0]
    sender_win_percentage = (sender_wins / total_games) * 100 if total_games > 0 else 0
    win_percentages.append((g, sender_win_percentage))

# print win percentages by sender
win_percentages.sort(key=lambda x: x[1], reverse=True)
print('Sender Win Percentages:')
for g, sender_win_percentage in win_percentages:
    print(f'{g}: {sender_win_percentage:.2f}%')


In [ ]:
# set save1
save1 = True

# Calculate the number of series won per player
aidan_wins = []
oliver_wins = []
winner = []
sender = []
aidan_winsbyseries = []
oliver_winsbyseries = []
drawsbyseries = []

for d in df['Date'].unique():
    series_df = df[df['Date'] == d]
    aidan_winsbyseries.append(len(series_df[series_df['Result'] == 'Aidan Win']))
    oliver_winsbyseries.append(len(series_df[series_df['Result'] == 'Oliver Win']))
    if len(series_df[series_df['Result'] == 'Aidan Win']) > len(series_df[series_df['Result'] == 'Oliver Win']):
        winner.append('Aidan')
    else:
        winner.append('Oliver')
    drawsbyseries.append(len(series_df[series_df['Result'] == 'Draw']))
    if 'Aidan' in series_df['Sender'].unique():
        sender.append('Aidan')
    else:
        sender.append('Oliver')
    a_wins = series_df[series_df['Result'] == 'Aidan Win'].shape[0]
    o_wins = series_df[series_df['Result'] == 'Oliver Win'].shape[0]
    if a_wins > o_wins:
        aidan_wins.append('win')
    else:
        oliver_wins.append('win')

sends_df = pd.DataFrame({'Sender': sender, 'Winner': winner})

title = f'Average Series Score:\nAidan {(np.mean(aidan_winsbyseries)):.2f}–{(np.mean(oliver_winsbyseries)):.2f} Oliver ({(np.mean(drawsbyseries)):.2f} Draws)'


fig, ax = plt.subplots(3, 1, figsize=(7, 6), sharey=False)

# Plot the win pct plot
byseriescount = [len(aidan_wins), 0, len(oliver_wins)]
bygamecount = [len(df[df['Result'] == 'Aidan Win']), len(df[df['Result'] == 'Draw']), len(df[df['Result'] == 'Oliver Win'])]

byseriescount_sum = sum(byseriescount)
bygamecount_sum = sum(bygamecount)
byseriescount_normalized = []
bygamecount_normalized = []

for i in range(3):
    if i == 0:
        byseriescount_normalized.append(byseriescount[i] / byseriescount_sum)
        bygamecount_normalized.append(bygamecount[i] / bygamecount_sum)
    elif i == 1:
        byseriescount_normalized.append((byseriescount[i-1] + byseriescount[i]) / byseriescount_sum)
        bygamecount_normalized.append((bygamecount[i-1] + bygamecount[i]) / bygamecount_sum)
    else:
        byseriescount_normalized.append(1)
        bygamecount_normalized.append(1)

ratios = byseriescount_normalized + bygamecount_normalized
percent = []
for i in ratios:
    percent.append(i * 100)

dataframe = pd.DataFrame({'Percent': percent, 'Result': ['Aidan Win', 'Draw', 'Oliver Win'] * 2, 'Category': ['Series'] * 3 + ['Game'] * 3})

colors = ['blue', '#add8e6', 'red']
sns.set_palette(colors)
invertedresult = ['Oliver Win', 'Draw', 'Aidan Win']

for i, x in enumerate(invertedresult):
    sns.barplot(y='Category', x='Percent', data=dataframe[dataframe['Result'] == x], legend=False, ax=ax[0], color=colors[i]).set_xlim(0, 100)
ax[0].set_title('Win Percentage by Series and Game')
ax[0].set_xlabel('Percentage')
ax[0].set_ylabel('')
ax[0].set_xticks([0, 25, 1/3*100, 50, 2/3*100, 75, 100])
ax[0].xaxis.grid(True)

print(f'Series sent by Aidan:\n    Aidan {sends_df[(sends_df['Sender'] == 'Aidan') & (sends_df['Winner'] == 'Aidan')].shape[0]}–{sends_df[(sends_df['Sender'] == 'Aidan') & (sends_df['Winner'] == 'Oliver')].shape[0]} Oliver')
print(f'Series sent by Oliver:\n    Aidan {sends_df[(sends_df['Sender'] == 'Oliver') & (sends_df['Winner'] == 'Aidan')].shape[0]}–{sends_df[(sends_df['Sender'] == 'Oliver') & (sends_df['Winner'] == 'Oliver')].shape[0]} Oliver')
print(f'Total Series Wins:\n    Aidan {len(aidan_wins)}–{len(oliver_wins)} Oliver')
print(f'Total Game Wins:\n    Aidan {len(df[df['Result'] == 'Aidan Win'])}–{len(df[df['Result'] == 'Oliver Win'])} Oliver ({len(df[df['Result'] == 'Draw'])} Draws)')

sns.set_palette(['red', 'blue', '#add8e6'])
hue_order = ['Aidan Win', 'Oliver Win', 'Draw']

b = sns.barplot(y='Date', x='Game', hue='Result', hue_order=hue_order, data=bargroup, ax=ax[2], legend=False)
b.set(ylabel='Count', xlabel='')
b.set_xticklabels(b.get_xticklabels(), rotation=90)
b.yaxis.grid(True)
p = sns.pointplot(y='Game', x='Date', hue='Result', hue_order=hue_order, data=pointgroup, ax=ax[1], markers='.', legend=True)
sns.pointplot(y='Game', x='Date', hue='Result', hue_order=hue_order, data=pointgroupnodraw, ax=ax[1], markers='.', legend=False)
p.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Result')
p.set(xlabel='', ylabel='Count')
p.set_ylim(-.5, np.max(pointgroup['Game'])+.5)
p.yaxis.grid(True)
p.set_xticklabels('')
p.set_xticks([])
# p.set_yticks([])
b.set_title('Results by Game')
p.set_title('Series Results Over Time')
ax[1].yaxis.set_major_locator(plt.MaxNLocator(integer=True))
# ax[2].yaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.suptitle(title)
plt.tight_layout()
if save1:
    plt.savefig('1Series.png', dpi=600)

plt.show()

In [ ]:
# set save2
save2 = True

sns.set_palette(['red', 'blue', '#add8e6'])
hue_order = ['Aidan Win', 'Oliver Win', 'Draw']

# Get the unique games
games = ['8 Ball', 'Archery', 'Darts', 'Cup Pong', 'Mini Golf', 'Knockout', 'Shuffleboard', '9 Ball', 'Tanks']

# Set up the figure and axes
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(7, 6), sharex=True, sharey=True)
axes = axes.flatten()

# Loop through each game and create a plot
for i, game in enumerate(games):
    game_df = df[df['Game'] == game]
    
    # Create a cumulative count of wins for each player
    game_df['Aidan Win'] = (game_df['Result'] == 'Aidan Win').cumsum()
    game_df['Oliver Win'] = (game_df['Result'] == 'Oliver Win').cumsum()
    game_df['Draw'] = (game_df['Result'] == 'Draw').cumsum()
    # Convert the Date column to string to ensure the space between dates is even
    game_df['Date'] = game_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # Melt the dataframe for easier plotting
    melted_df = game_df.melt(id_vars=['Date'], value_vars=['Aidan Win', 'Oliver Win', 'Draw'], 
                             var_name='Player', value_name='Cumulative Wins')
    
    nodraws = [0, 3, 7, 8]

    # Create the line plot
    if i == 5:
        p = sns.lineplot(x='Date', y='Cumulative Wins', hue='Player', hue_order=hue_order, data=melted_df, markers='o', ax=axes[i], legend=True)
        sns.lineplot(x='Date', y='Cumulative Wins', hue='Player', data=melted_df[melted_df['Player'] != 'Draw'], markers='o', ax=axes[i], legend=False)
    elif i in nodraws:
        sns.lineplot(x='Date', y='Cumulative Wins', hue='Player', hue_order=hue_order, data=melted_df[melted_df['Player'] != 'Draw'], markers='o', ax=axes[i], legend=False)
    else:
        sns.lineplot(x='Date', y='Cumulative Wins', hue='Player', hue_order=hue_order, data=melted_df, markers='o', ax=axes[i], legend=False)
        sns.lineplot(x='Date', y='Cumulative Wins', hue='Player', data=melted_df[melted_df['Player'] != 'Draw'], markers='o', ax=axes[i], legend=False)
    axes[i].set_title(f'{game}')
    axes[i].set_xticks([])
    axes[i].set_xlabel('Series')
    axes[i].set_ylabel('Result (Cumulative)')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].set_xticklabels('')
    axes[i].yaxis.grid(True)

p.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Result')

plt.suptitle('Game Results Over Time')


plt.tight_layout()
if save2:
    plt.savefig('2Series_Cumulative_Wins.png', dpi=600)
plt.show()

In [ ]:
# set save3
save3 = True

results = df['Result'].unique()
resultaidan = []
resultoliver = []

for d in df['Date'].unique():
    x = df[df['Date'] == d]
    for i, y in enumerate(results):
        if y not in x['Result'].unique():
            if y == 'Aidan Win':
                resultaidan.append(0)
            elif y == 'Oliver Win':
                resultoliver.append(0)
        else:
            if y == 'Aidan Win':
                resultaidan.append(len(x[x['Result'] == y]))
            elif y == 'Oliver Win':
                resultoliver.append(len(x[x['Result'] == y]))
resultbyplayer = pd.DataFrame({'Aidan Game Wins': resultaidan, 'Oliver Game Wins': resultoliver})

resultbyplayer['Winner Margin'] = resultbyplayer['Aidan Game Wins'] - resultbyplayer['Oliver Game Wins']
resultbyplayer['Series Winner'] = resultbyplayer.apply(lambda row: 'Aidan' if row['Winner Margin'] > 0 else ('Oliver' if row['Winner Margin'] < 0 else 'Draw'), axis=1)
resultbyplayer['Frequency'] = resultbyplayer.groupby(['Aidan Game Wins', 'Oliver Game Wins'])['Series Winner'].transform('count')

# print(resultbyplayer)

unique_combinations = resultbyplayer.groupby(['Aidan Game Wins', 'Oliver Game Wins']).size().reset_index(name='Frequency')

unique_combinations['Series Winner'] = unique_combinations.apply(lambda row: 'Aidan' if row['Aidan Game Wins'] > row['Oliver Game Wins'] else ('Oliver' if row['Aidan Game Wins'] < row['Oliver Game Wins'] else 'Draw'), axis=1)

unique_combinations['Winner Game Wins'] = unique_combinations.apply(lambda row: row['Aidan Game Wins'] if row['Series Winner'] == 'Aidan' else (row['Oliver Game Wins'] if row['Series Winner'] == 'Oliver' else 0), axis=1)
unique_combinations['Loser Game Wins'] = unique_combinations.apply(lambda row: row['Oliver Game Wins'] if row['Series Winner'] == 'Aidan' else (row['Aidan Game Wins'] if row['Series Winner'] == 'Oliver' else 0), axis=1)

unique_combinations['Aidan-Oliver Result'] = unique_combinations['Aidan Game Wins'].astype(str) + '-' + unique_combinations['Oliver Game Wins'].astype(str)
unique_combinations['Score Sum'] = unique_combinations['Aidan Game Wins'] + unique_combinations['Oliver Game Wins']
unique_combinations['Margin'] = unique_combinations['Aidan Game Wins'] - unique_combinations['Oliver Game Wins']
unique_combinations['Margin*Score Sum'] = unique_combinations['Margin'] * unique_combinations['Score Sum']
unique_combinations['Abs Score'] = unique_combinations.apply(lambda row: row['Aidan-Oliver Result'][::-1] if row['Margin'] < 0 else row['Aidan-Oliver Result'], axis=1)
unique_combinations = unique_combinations.sort_values(by=['Frequency', 'Margin', 'Score Sum'], ascending=[False, False, True])
unique_combinations.reset_index(drop=True, inplace=True)
# unique_combinations = unique_combinations.drop(['Aidan Game Wins', 'Oliver Game Wins'], axis=1)
# unique_combinations = unique_combinations.iloc[:, [1, 2, 3, 4, 0, 5]]
unique_combinations.index += 1

fig, ax = plt.subplots(figsize=(7, 6))

x_marks = [i for i in range(9)] * 9
y_marks = (pd.Series(x_marks[:9])).repeat(9)
d_marks = pd.DataFrame({'x': x_marks, 'y': y_marks})
d_marks = d_marks[d_marks['x'] != d_marks['y']]
d_marks.reset_index(drop=True, inplace=True)

deleteindex = []
deletepairs = [[8, 0], [7, 1], [6, 2], [5, 4]]

# comment the block below to show all ticks
d_marks_possible = d_marks[d_marks['x'] + d_marks['y'] > 2]
for d in d_marks.index:
    for p in deletepairs:
        for i, x in enumerate([['x', 'y'], ['y', 'x']]):
            if d_marks[x[0]][d] == p[0] and d_marks[x[1]][d] > p[1]:
                deleteindex.append(d)
d_marks_possible = d_marks_possible.drop(deleteindex)


# p_marks shows all possible marks except x=y
# p_marks = sns.scatterplot(data=d_marks, x='x', y='y', marker='+', color='gray', alpha=.25, legend=False, ax=ax)

# p_marks_possible only shows marks at possible final scores
p_marks_possible = sns.scatterplot(data=d_marks_possible, x='x', y='y', marker='+', color='k', legend=False, ax=ax)

g = sns.scatterplot(
    data=resultbyplayer, 
    y='Aidan Game Wins', 
    x='Oliver Game Wins', 
    hue='Series Winner', 
    hue_order=['Aidan', 'Oliver'], 
    palette=['red', 'blue'], 
    size='Frequency', 
    legend=True,
    sizes=(50, 500),
    ax=ax
)
fig.suptitle('Series Result Frequency')
g.plot([0, 8], [0, 8], color='k', linestyle='--', linewidth=1, alpha=0.5)
g.set_xlim(-.5, 8.5)
g.set_ylim(-.5, 8.5)
g.set_xlabel('Oliver Game Wins')
g.set_ylabel('Aidan Game Wins')
g.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
if save3:
    plt.savefig('3Result Frequency.png', dpi=600)

plt.show()


In [ ]:
# set save4
save4 = True

unique_combinations = unique_combinations.sort_values(by=['Winner Game Wins', 'Loser Game Wins'], ascending=[True, False])

fig, ax = plt.subplots(figsize=(7, 6))

g = sns.barplot(data=unique_combinations, x='Frequency', y='Abs Score', hue='Series Winner', hue_order=['Aidan', 'Oliver'], palette=['red', 'blue'], legend=True, ax=ax)
g.set_ylabel('Series Result')
g.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Series Winner')

plt.suptitle('Series Result Frequency')
plt.tight_layout()

if save4 == True:
    plt.savefig('4Result Histogram.png', dpi=600)

plt.show()

In [ ]:
pd.set_option('future.no_silent_downcasting', True)

resultbydate = pd.DataFrame()

for d in df['Date'].unique():
    temp_df = df[df['Date'] == d]
    if len(temp_df) == 8:
        temp_df = pd.concat([temp_df, pd.DataFrame([[0]*5], columns=temp_df.columns)], ignore_index=True)
    # print(temp_df)
    temp_df.replace({'Result': ['Aidan Win', 'Oliver Win', 'Draw']}, {'Result': [1, -1, 0]}, inplace=True)
    # print(temp_df)
    resultbydate[d] = temp_df['Result']

# winner = []

for c in resultbydate.columns:
    
    winner.append(resultbydate[c].sum())

# print(winner)

print(resultbydate[resultbydate.columns[9]])

In [ ]:
df

In [ ]:
# LIVE ODDS

# ENTER RESULTS
current_sender = 'Oliver'
games_played_result = []

# create df_sender
if current_sender is None:
    df_sender = df
else:
    df_sender = df[df['Sender'] == current_sender]

# instantiate series_winners list to store values
series_winners = []

# loop through each date in df_sender and add 1 to series_winners if the series played out like the current series and Aidan won, and add 0 if Oliver won
for d in df_sender['Date'].unique():
    temporary_df = df_sender[df_sender['Date'] == d]
    temporary_df.set_index('Game', inplace=True)
    match = []
    for i, x in enumerate(games_played_result):
        game_slice = temporary_df.loc[x[0]]
        if game_slice['Result'] == x[1]:
            match.append(True)
        else:
            match.append(False)
    if all(match):
        if (temporary_df['Result'] == 'Aidan Win').sum() > (temporary_df['Result'] == 'Oliver Win').sum():
            series_winners.append(1)
        else:
            series_winners.append(0)

# print results
if len(series_winners) == 0:
    print('No series have played out like this yet')
else:
    print(f'Number of similar series: {len(series_winners)}')
    print('Series win probability:')
    print(f'Aidan: {np.mean(series_winners)*100:.0f}%')
    print(f'Oliver: {(1-np.mean(series_winners))*100:.0f}%')